In [10]:
import requests
from barentswatch.credentials import config
from barentswatch.authentication import get_token
import pandas as pd
import time

#### Functions for extracting yearly data


In [11]:

def get_locality(config, token, year, week):
    """Get data about locality for a given year and week using /v1/geodata/fishhealth/locality/{year}/{week}

    Args:
        year (int): year to extract data
        week (int): week to extract data

    Returns:
        list: list of dictionaries containing the locality data
    """
    url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{year}/{week}"
    headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',}
    r = requests.get(url, headers=headers)
    return r.json()   

In [12]:
def get_week_summary(token, year, week):
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{year}/{week}"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()
token = get_token()

Token request successful


In [15]:
#Connecting to cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect() #Kan vel være et argument til funksjonen? Tenkte på dette

session.set_keyspace('fish_data')

In [16]:
import os

os.environ["PYSPARK_PYTHON"] = r"C:\Users\holml\anaconda3\envs\IND320_ml\python.exe" 
os.environ["PYSPARK_DRIVER_PYTHON"] = r"C:\Users\holml\anaconda3\envs\IND320_ml\python.exe"
os.environ["HADOOP_HOME"] = r"C:\Source\winutils\hadoop-3.3.1"
os.environ["PYSPARK_HADOOP_VERSION"] = r"C:\Source\winutils\hadoop-3.3.1"

In [17]:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.appName('SparkCassandraApp').\
        config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.1').\
        config('spark.cassandra.connection.host', 'localhost').\
        config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
        config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
        config("spark.cassandra.output.batch.size.rows", "1000").\
        config('spark.cassandra.connection.port', '9042').getOrCreate()

In [23]:
#Check if year is present in fish_space.localitites
def check_year(year):
    query = f"SELECT * FROM fish_data.localities WHERE year = {year} ALLOW FILTERING"
    rows = session.execute(query)
    if rows:
        return True
    else:
        return False
check_year(2018)

False

In [18]:
### Checking if the year already exist in the cassandra database
insert_query = """
    INSERT INTO localities (year, week, localityNo, localityWeekId, name, hasReportedLice, isFallow, 
                                avgAdultFemaleLice, hasCleanerfishDeployed, hasMechanicalRemoval, 
                                hasSubstanceTreatments, hasPd, hasIla, municipalityNo, municipality, 
                                lat, lon, isOnLand, inFilteredSelection, hasSalmonoids, isSlaughterHoldingCage)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [ ]:
for week in range(1, 53):
    print(f'Inserting data for week {week}...')
    weeksummary = get_week_summary(token, year, str(week))
    for locality in weeksummary['localities']:
        session.execute_async(
            insert_query,
            (
                weeksummary['year'],
                weeksummary['week'],
                locality['localityNo'],
                locality['localityWeekId'],
                locality['name'],
                locality['hasReportedLice'],
                locality['isFallow'],
                locality['avgAdultFemaleLice'],
                locality['hasCleanerfishDeployed'],
                locality['hasMechanicalRemoval'],
                locality['hasSubstanceTreatments'],
                locality['hasPd'],
                locality['hasIla'],
                int(locality['municipalityNo']),
                locality['municipality'],
                locality['lat'],
                locality['lon'],
                locality['isOnLand'],
                locality['inFilteredSelection'],
                locality['hasSalmonoids'],
                locality['isSlaughterHoldingCage']
            )
        )

Inserting data for week 1...
1
Inserting data for week 2...
2


23/11/22 22:20:25 WARN ControlConnection: [s0] Error connecting to Node(endPoint=localhost/127.0.0.1:9042, hostId=7e746b43-170e-4665-8e02-13458406ab69, hashCode=4a21c6cb), trying next node (DriverTimeoutException: [s0|control|id: 0x25f513e6, L:/127.0.0.1:60166 - R:localhost/127.0.0.1:9042] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=92dc7fdd-3b66-46dd-a742-b2a68f67dd92, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700669074747}): timed out after 5000 ms)
23/11/22 22:20:25 WARN ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (DriverTimeoutException: [s0|id: 0xa0dafd49, L:/127.0.0.1:60102 - R:localhost/127.0.0.1:9042] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=92dc7fdd-3b66-46dd-a742-b2a68f67dd92, APPLICATION_NAME=Spark-Cassan

KeyboardInterrupt: 

Traceback (most recent call last):
  File "cassandra/cluster.py", line 3541, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra/cluster.py", line 3563, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra/cluster.py", line 1630, in cassandra.cluster.Cluster.connection_factory
  File "cassandra/connection.py", line 859, in cassandra.connection.Connection.factory
cassandra.OperationTimedOut: errors=Timed out creating connection (5 seconds), last_host=None
23/11/22 22:23:01 WARN ChannelPool: [s0|localhost/127.0.0.1:9042]  Error while opening new channel (DriverTimeoutException: [s0|id: 0x325275bf, L:/127.0.0.1:56452 - R:localhost/127.0.0.1:9042] Protocol initialization request, step 1 (STARTUP {CQL_VERSION=3.0.0, DRIVER_NAME=DataStax Java driver for Apache Cassandra(R), DRIVER_VERSION=4.13.0, CLIENT_ID=92dc7fdd-3b66-46dd-a742-b2a68f67dd92, APPLICATION_NAME=Spark-Cassandra-Connector-local-1700669074747}): timed out after 5000 ms)
23/11/22 22:23:01

Traceback (most recent call last):
  File "cassandra/cluster.py", line 3541, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra/cluster.py", line 3563, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra/cluster.py", line 1630, in cassandra.cluster.Cluster.connection_factory
  File "cassandra/connection.py", line 859, in cassandra.connection.Connection.factory
cassandra.OperationTimedOut: errors=Timed out creating connection (5 seconds), last_host=None
Traceback (most recent call last):
  File "cassandra/cluster.py", line 3541, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra/cluster.py", line 3563, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra/cluster.py", line 1630, in cassandra.cluster.Cluster.connection_factory
  File "cassandra/connection.py", line 859, in cassandra.connection.Connection.factory
cassandra.OperationTimedOut: errors=Timed out creating connection (5 seconds), last_host=Non

In [24]:
#Take time the cell use to run and print in the end
#Print the time used so far in seconds after print(week)
year = 2018
start_time = time.time()
list_localitites = []
year_in_db = check_year(year)
from functions import get_week_summary

if year_in_db:
    pass
else:
    for week in range(1, 53):
        start_it = time.time() 

        locality_data = get_week_summary(token, year, week)
        for row in locality_data["localities"]:
            row["year"] = locality_data["year"]
            row["week"] = locality_data["week"]

            dt = pd.Timestamp.fromisocalendar(locality_data["year"], locality_data["week"], 1)
            dt = dt.tz_localize('UTC')
            row["datetime"] = dt.strftime('%Y-%m-%d %H:%M:%S')
            list_localitites.append(row)
        print(f"Week: {week:>2}--- %s seconds" % (time.time() - start_it))
        df_localities_year = pd.DataFrame(list_localitites)
        df_localities_year.columns = map(str.lower, df_localities_year.columns)
        #Make all columns name small letters
        
    print(df_localities_year.shape)

    print("Total time: %s seconds ---" % (time.time() - start_time))

Week:  1--- 0.7958188056945801 seconds
Week:  2--- 0.5920321941375732 seconds
Week:  3--- 0.7669072151184082 seconds
Week:  4--- 0.6881039142608643 seconds
Week:  5--- 0.7927930355072021 seconds
Week:  6--- 0.7347080707550049 seconds
Week:  7--- 0.6335101127624512 seconds
Week:  8--- 0.6371841430664062 seconds
Week:  9--- 0.7171177864074707 seconds
Week: 10--- 0.6663081645965576 seconds
Week: 11--- 0.5716259479522705 seconds
Week: 12--- 0.5859527587890625 seconds
Week: 13--- 0.6941020488739014 seconds
Week: 14--- 0.7057192325592041 seconds
Week: 15--- 0.6243369579315186 seconds
Week: 16--- 0.6132721900939941 seconds
Week: 17--- 0.533735990524292 seconds
Week: 18--- 0.675762414932251 seconds
Week: 19--- 0.768423318862915 seconds
Week: 20--- 0.7453799247741699 seconds
Week: 21--- 0.6051833629608154 seconds
Week: 22--- 0.7376117706298828 seconds
Week: 23--- 0.6377501487731934 seconds
Week: 24--- 0.6657505035400391 seconds
Week: 25--- 0.5844776630401611 seconds
Week: 26--- 0.57877802848815

In [ ]:
insert_stmt = session.prepare("""
    INSERT INTO localities (year, week, localityNo, localityWeekId, name, hasReportedLice, isFallow, 
                                avgAdultFemaleLice, hasCleanerfishDeployed, hasMechanicalRemoval, 
                                hasSubstanceTreatments, hasPd, hasIla, municipalityNo, municipality, 
                                lat, lon, isOnLand, inFilteredSelection, hasSalmonoids, isSlaughterHoldingCage)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
""")

# Batch insert
batch = BatchStatement()
for index, row in df_localities_year.iterrows():
    print(index)
    batch.add(insert_stmt, (row['col1'], row['col2'], ...))
    if len(batch) >= 100:  # Adjust batch size as needed
        session.execute(batch)
        batch.clear()

# Execute any remaining queries
if len(batch) > 0:
    session.execute(batch)

In [25]:
df_localities_year.head()

,localityno,localityweekid,name,hasreportedlice,isfallow,avgadultfemalelice,hascleanerfishdeployed,hasmechanicalremoval,hassubstancetreatments,haspd,...,municipality,lat,lon,isonland,infilteredselection,hassalmonoids,isslaughterholdingcage,year,week,datetime
0,14746,713474,Aarsand,False,True,NaN,False,False,False,False,...,Bindal,65.045867,12.156933,False,True,False,False,2018,1,2018-01-01 00:00:00
1,31937,717244,Abelsnes,False,True,NaN,False,False,False,False,...,Flekkefjord,58.238767,6.656650,True,True,False,False,2018,1,2018-01-01 00:00:00
2,10665,545132,Adamselv,False,True,NaN,False,False,False,False,...,Lebesby,70.408000,26.691333,True,True,True,False,2018,1,2018-01-01 00:00:00
3,29196,837130,Adjetjohka,False,True,NaN,False,False,False,False,...,Guovdageaidnu-Kautokeino,68.944137,22.918715,True,True,False,False,2018,1,2018-01-01 00:00:00
4,13823,545479,Ådlandsvatn,False,True,NaN,False,False,False,False,...,Stord,59.791350,5.500367,True,True,True,False,2018,1,2018-01-01 00:00:00


your 131072x1 screen size is bogus. expect trouble
23/11/22 17:04:28 WARN Utils: Your hostname, Lenovo-Ivar resolves to a loopback address: 127.0.1.1; using 172.18.175.46 instead (on interface eth0)
23/11/22 17:04:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ivholmlu/miniconda3/envs/ind320_ml/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ivholmlu/.ivy2/cache
The jars for the packages stored in: /home/ivholmlu/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-342dac16-0bd1-408a-b85f-e0cba8ee8ab1;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0

In [ ]:
df_localities_year.head()

,localityno,localityweekid,name,hasreportedlice,isfallow,avgadultfemalelice,hascleanerfishdeployed,hasmechanicalremoval,hassubstancetreatments,haspd,...,municipality,lat,lon,isonland,infilteredselection,hassalmonoids,isslaughterholdingcage,year,week,datetime
0,14746,713526,Aarsand,False,True,NaN,False,False,False,False,...,Bindal,65.045867,12.156933,False,True,False,False,2019,1,2018-12-31 00:00:00
1,31937,717296,Abelsnes,False,True,NaN,False,False,False,False,...,Flekkefjord,58.238767,6.656650,True,True,False,False,2019,1,2018-12-31 00:00:00
2,10665,645029,Adamselv,False,True,NaN,False,False,False,False,...,Lebesby,70.408000,26.691333,True,True,True,False,2019,1,2018-12-31 00:00:00
3,29196,837182,Adjetjohka,False,True,NaN,False,False,False,False,...,Guovdageaidnu-Kautokeino,68.944137,22.918715,True,True,False,False,2019,1,2018-12-31 00:00:00
4,13823,645374,Ådlandsvatn,False,True,NaN,False,False,False,False,...,Stord,59.791350,5.500367,True,True,True,False,2019,1,2018-12-31 00:00:00


In [26]:
#append pd dataframe to cassandra table using pyspark
spark_df_localities_year = spark.createDataFrame(df_localities_year)

In [27]:
spark_df_localities_year.write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="localities", keyspace="fish_data")\
    .save()

In [32]:
# Show fish_data.localities
spark_df_localities_year = spark.read.format("org.apache.spark.sql.cassandra").options(table="localities", keyspace="fish_data").load()
pd_df_localities_year = spark_df_localities_year.toPandas()
pd_df_localities_year.shape
#Save dataframe to csv
pd_df_localities_year.to_csv(r'C:\Users\holml\OneDrive\Documents\GitHub\IND320_ML\data\localities.csv', index = False)

OSError: Cannot save file into a non-existent directory: 'C:\Users\holml\OneDrive\Documents\GitHub\IND320\ca4'

In [34]:
pd_df_localities_year.to_csv(r'data\localities.csv', index = False)

In [30]:
#Show size of spark dataframe
unique_values = spark_df_localities_year.select('year').distinct()
unique_values.show()

+----+
|year|
+----+
|2018|
|2019|
|2020|
+----+



In [ ]:
spark_df_localities_year = spark.read.format("org.apache.spark.sql.cassandra").options(table="localities", keyspace="fish_data").load()